# 1. Configurar ambiente


In [8]:
import pandas as pd
import torch
import pickle
import jellyfish
import re
import unicodedata
import spacy
from transformers import BertForSequenceClassification
from transformers import BertTokenizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


# 2.Normalizando Dialogos

In [9]:
# Importando dataset
path = '../data/df_dialogo.csv'
df_dialogo = pd.read_csv(path)
df_dialogo.head()

,dialogo,respuesta,tipo,interseccion,jaro_winkler,probabilidad
0,ahora quien responde,Yo respondo.,Identidad,0,0,0
1,ahora ya no se ni con quien o que estoy hablando,"Lo siento, no entendí tu pregunta. ¿Puedes exp...",Identidad,0,0,0
2,ahorita estoy chateando con una ia,¡Sí soy una Inteligencia Artificial llamada Ma...,Identidad,0,0,0
3,bien quien eres,"¡Hola! Me llamo Madre, ¡el chatbot más guay de...",Identidad,0,0,0
4,el chat bot o la persona,¡Sí soy una Inteligencia Artificial llamada Ma...,Identidad,0,0,0


## 2.1 Funciones Tratamiento Texto

In [11]:
url_lista_verbos = "../models/lista_verbos.pickle"
url_verbos_irregulares = "../models/verbos_irregulares.pickle"

# Función para cargar archivos pickle desde una ruta local
def cargar_pickle_desde_archivo(ruta):
    with open(ruta, 'rb') as file:
        return pickle.load(file)

# Cargar los archivos en variables
lista_verbos = cargar_pickle_desde_archivo(url_lista_verbos)
verbos_irregulares = cargar_pickle_desde_archivo(url_verbos_irregulares)

# Mostrar resultados
print(lista_verbos)
print(verbos_irregulares)


['parar', 'recomendar', 'cancelar', 'fanatizar', 'amaran o amasen', 'exponer', 'obedecer', 'quejar', 'echar', 'legitimar', 'perjudicar', 'organizar', 'molar', 'objetar', 'considerar', 'golear', 'mover', 'acertar', 'reunir', 'regir', 'ilusionar', 'simpatizar', 'conjeturar', 'helar', 'quitar', 'amariamos', 'destacar', 'llegar', 'sincronizar', 'lesionar', 'seducir', 'asistir', 'conservar', 'acordar', 'salvar', 'relucir', 'graduar', 'forzar', 'dar', 'deplorar', 'batear', 'mofar', 'estropear', 'aplastar', 'wasapeo', 'gestionar', 'suprimir', 'gruñir', 'progresar', 'suscribir', 'noticiar', 'cavar', 'alejar', 'galopar', 'virar', 'medir', 'actualizar', 'humanizar', 'convivir', 'gratificar', 'digerir', 'tocar', 'zonificar', 'amariais', 'aterrizar', 'hojear', 'cometer', 'sufrir', 'reciclar', 'obturar', 'divertir', 'ondear', 'listar', 'determinar', 'alentar', 'sumar', 'reflexionar', 'ames', 'anotar', 'mitificar', 'escribir', 'ilustrar', 'obtener', 'subir', 'socorrer', 'desprender', 'agregar', 'gan

In [12]:
# Función para reemplazar el final de una palabra por 'r'
def reemplazar_terminacion(palabra):
    terminaciones = ["ste","es", "me", "as", "te"]

    for terminacion in terminaciones:
        if palabra.endswith(terminacion):
            return palabra[:-len(terminacion)] + "r"

    return palabra  # Si no coincide, devuelve la palabra original

#Función para encontrar la raiz de las palabras
def raiz(palabra):
  max_similaridad = 0
  mejor_coincidencia = palabra #Por defecto, devolvemos la palabra original

  for verbo in lista_verbos:
    similitud = jellyfish.jaro_winkler_similarity(palabra, verbo)
    if similitud > max_similaridad:
      max_similaridad = similitud
      mejor_coincidencia = verbo

  return mejor_coincidencia if max_similaridad >= 0.93 else palabra

def tratamiento_texto(texto):
    # Convertir a minúsculas
    texto = texto.lower()

    # Eliminar acentos
    texto = ''.join(c for c in unicodedata.normalize('NFD', texto) if unicodedata.category(c) != 'Mn')

    # Eliminar espacios adicionales
    texto = re.sub(r'\s+', ' ', texto).strip()

    return texto


In [13]:
#Función para devolver los tokens normalizados del texto
nlp = spacy.load('es_core_news_md')
def normalizar(texto):
  tokens=[]
  doc = nlp(texto)
  for t in doc:
    lemma=verbos_irregulares.get(t.text, t.lemma_.split()[0])
    lemma=re.sub(r'[^\w\s+\-*/]', '', lemma)
    if t.pos_ in ('VERB','PROPN','PRON','NOUN','AUX','SCONJ','ADJ','ADV','NUM') or lemma in lista_verbos:
      if t.pos_=='VERB':
        lemma = reemplazar_terminacion(lemma)
        tokens.append(raiz(tratamiento_texto(lemma)))
      else:
        tokens.append(tratamiento_texto(lemma))

  tokens = list(dict.fromkeys(tokens))
  tokens = list(filter(None, tokens))
  return tokens

## 2.2 Normalizando las frases

In [14]:
#Normalizando las frases
label_encoder = LabelEncoder()
df_dialogo['palabras'] = df_dialogo['dialogo'].apply(lambda x: ' '.join(normalizar(x)))
df_dialogo['tipo_num'] = label_encoder.fit_transform(df_dialogo['tipo'])
df_dialogo = df_dialogo[df_dialogo.palabras.values!='']
df_dialogo

,dialogo,respuesta,tipo,interseccion,jaro_winkler,probabilidad,palabras,tipo_num
0,ahora quien responde,Yo respondo.,Identidad,0,0,0,ahora quien responder,8
1,ahora ya no se ni con quien o que estoy hablando,"Lo siento, no entendí tu pregunta. ¿Puedes exp...",Identidad,0,0,0,ahora ya no el quien que estar hablar,8
2,ahorita estoy chateando con una ia,¡Sí soy una Inteligencia Artificial llamada Ma...,Identidad,0,0,0,ahorita estar chatear ia,8
3,bien quien eres,"¡Hola! Me llamo Madre, ¡el chatbot más guay de...",Identidad,0,0,0,bien quien ser,8
4,el chat bot o la persona,¡Sí soy una Inteligencia Artificial llamada Ma...,Identidad,0,0,0,chat bot persona,8
...,...,...,...,...,...,...,...,...
1119,existe algun otro canal de comunicacion para h...,"En este momento, la plataforma de chat es el ú...",Contacto,0,0,0,existir canal comunicacion hablar tu,2
1120,puedo encontrarte en redes sociales como faceb...,"No tengo presencia en las redes sociales, por ...",Contacto,0,0,0,poder encontrar red social como facebook twitter,2
1121,tienes un numero de telefono al que pueda llam...,"Como chatbot, no tengo un número de teléfono a...",Contacto,0,0,0,tener numero telefono que poder llamar hablar tu,2
1122,hay alguna aplicacion movil para interactuar c...,"Actualmente, no hay una aplicación móvil dedic...",Contacto,0,0,0,haber aplicacion movil interactuar tu,2


## 2.3 Diccionario de relacion

In [15]:
# Imprimir diccionario
relacion_diccionario = {}

# Iterar sobre las filas del DataFrame
for tipo, tipo_num in zip(df_dialogo['tipo'], df_dialogo['tipo_num']):
    relacion_diccionario[tipo_num] = tipo

# Imprimir el diccionario
print(relacion_diccionario)

{8: 'Identidad', 9: 'Nombre', 4: 'Despedida', 7: 'Funcion', 0: 'Agradecimiento', 14: 'Usuario', 13: 'Sentimiento', 10: 'Origen', 3: 'Continuacion', 12: 'Saludos', 6: 'Error', 1: 'Aprendizaje', 11: 'Otros', 5: 'Edad', 2: 'Contacto'}


# 3.Entrenando con Naive Bayes

In [16]:
# Separar los datos en características (X) y etiquetas (y)
X = df_dialogo['palabras']
y = df_dialogo['tipo_num']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorizar los datos de texto
vectorizer = CountVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

# Entrenar el clasificador de Naive Bayes
modelo_NB = MultinomialNB()
modelo_NB.fit(X_train_vect, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = modelo_NB.predict(X_test_vect)

# Calcular el accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Precisión:", accuracy)

Precisión: 0.6607142857142857


In [ ]:
# Guardar el vectorizador y el modelo en un archivo
with open('modelo_naive_bayes.pickle', 'wb') as archivo:
    pickle.dump((vectorizer, modelo_NB), archivo)

# Cargar el vectorizador y el modelo desde el archivo
with open('modelo_naive_bayes.pickle', 'rb') as archivo:
    vectorizer, modelo_NB = pickle.load(archivo)

In [ ]:
# Calcular la precisión por clase
unique_classes = df_dialogo['tipo_num'].unique()
for cls in unique_classes:
    cls_indices = y_test == cls
    cls_accuracy = accuracy_score(y_test[cls_indices], y_pred[cls_indices])
    print("Accuracy para la clase", df_dialogo[df_dialogo.tipo_num == cls]['tipo'].unique()[0], ":", cls_accuracy)

Accuracy para la clase Identidad : 0.65
Accuracy para la clase Nombre : 0.0
Accuracy para la clase Despedida : 0.6
Accuracy para la clase Funcion : 0.5384615384615384
Accuracy para la clase Agradecimiento : 0.3076923076923077
Accuracy para la clase Usuario : 0.4166666666666667
Accuracy para la clase Sentimiento : 0.2
Accuracy para la clase Origen : 0.6666666666666666
Accuracy para la clase Continuacion : 0.6666666666666666
Accuracy para la clase Saludos : 0.8928571428571429
Accuracy para la clase Error : 0.3333333333333333
Accuracy para la clase Aprendizaje : 0.26666666666666666
Accuracy para la clase Otros : 0.890625
Accuracy para la clase Edad : 0.2
Accuracy para la clase Contacto : 0.0


In [ ]:
df_dialogo['tipo'].value_counts()

,count
tipo,
Otros,324
Saludos,180
Identidad,81
Funcion,73
Sentimiento,71
Agradecimiento,66
Despedida,54
Origen,50
Aprendizaje,43


In [ ]:
# Procesando la nueva frase
frase = ' '.join(normalizar('¿Qué lenguajes de programación se utilizan para crear un chatbot?'))
nueva_frase_vect = vectorizer.transform([frase])

# Realizar la predicción
prediccion = modelo_NB.predict(nueva_frase_vect)

diccionario = {0: 'Agradecimiento',1: 'Aprendizaje',2: 'Contacto',3: 'Continuacion',
               4: 'Despedida',5: 'Edad',6: 'Error',7: 'Funcion',8: 'Identidad',
               9: 'Nombre',10: 'Origen',11: 'Otros',12: 'Saludos',13: 'Sentimiento',
               14: 'Usuario'}

llave_buscada = prediccion[0]
clase_encontrada = diccionario[llave_buscada]

print("La frase", frase, "se clasifica como: ", clase_encontrada)

La frase lenguaje programacion el utilizar crear chatbot se clasifica como:  Origen


# 4.Entrenando con Random Forest

In [ ]:
# Separar los datos en características (X) y etiquetas (y)
X = df_dialogo['palabras']
y = df_dialogo['tipo_num']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorizar los datos de texto
vectorizer = CountVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

# Entrenar el clasificador Random Forest
Modelo_RF = RandomForestClassifier()
Modelo_RF.fit(X_train_vect, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = Modelo_RF.predict(X_test_vect)

# Calcular el accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Precisión:", accuracy)

Precisión: 0.6785714285714286


In [ ]:
# Guardar el vectorizador y el modelo en un archivo
with open('modelo_random_forest.pickle', 'wb') as archivo:
    pickle.dump((vectorizer, Modelo_RF), archivo)

# Cargar el vectorizador y el modelo desde el archivo
with open('modelo_random_forest.pickle', 'rb') as archivo:
    vectorizer, Modelo_RF = pickle.load(archivo)

In [ ]:
# Calcular la precisión por clase
unique_classes = df_dialogo['tipo_num'].unique()
for cls in unique_classes:
    cls_indices = y_test == cls
    cls_accuracy = accuracy_score(y_test[cls_indices], y_pred[cls_indices])
    print("Accuracy para la clase", df_dialogo[df_dialogo.tipo_num == cls]['tipo'].unique()[0], ":", cls_accuracy)

Accuracy para la clase Identidad : 0.95
Accuracy para la clase Nombre : 0.6666666666666666
Accuracy para la clase Despedida : 0.5
Accuracy para la clase Funcion : 0.7692307692307693
Accuracy para la clase Agradecimiento : 0.46153846153846156
Accuracy para la clase Usuario : 0.5
Accuracy para la clase Sentimiento : 0.4
Accuracy para la clase Origen : 0.6666666666666666
Accuracy para la clase Continuacion : 0.3333333333333333
Accuracy para la clase Saludos : 0.8571428571428571
Accuracy para la clase Error : 1.0
Accuracy para la clase Aprendizaje : 0.5333333333333333
Accuracy para la clase Otros : 0.78125
Accuracy para la clase Edad : 0.4
Accuracy para la clase Contacto : 0.2


In [ ]:
# Procesando la nueva frase
frase = ' '.join(normalizar('como haces para aprender tan rapido?'))
nueva_frase_vect = vectorizer.transform([frase])

# Realizar la predicción
prediccion = modelo_NB.predict(nueva_frase_vect)

diccionario = {0: 'Agradecimiento',1: 'Aprendizaje',2: 'Contacto',3: 'Continuacion',
               4: 'Despedida',5: 'Edad',6: 'Error',7: 'Funcion',8: 'Identidad',
               9: 'Nombre',10: 'Origen',11: 'Otros',12: 'Saludos',13: 'Sentimiento',
               14: 'Usuario'}

llave_buscada = prediccion[0]
clase_encontrada = diccionario[llave_buscada]

print("La frase", frase, "se clasifica como: ", clase_encontrada)

La frase como hacer aprender tanto rapido se clasifica como:  Otros


# 5.Entrenando con Transformers

In [ ]:
# Dividir los datos en conjunto de entrenamiento y conjunto de prueba
df_train, df_test = train_test_split(df_dialogo, test_size=0.2, random_state=42)

# Cargar el modelo preentrenado de BERT para clasificación en español
model_name = 'dccuchile/bert-base-spanish-wwm-uncased'
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=df_dialogo['tipo_num'].nunique())
tokenizer = BertTokenizer.from_pretrained(model_name)

# Tokenizar y codificar las frases de entrenamiento
train_inputs = tokenizer.batch_encode_plus(
    df_train['palabras'].tolist(),
    max_length=128,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)

# Tokenizar y codificar las frases de prueba
test_inputs = tokenizer.batch_encode_plus(
    df_test['palabras'].tolist(),
    max_length=128,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)

# Preparar los datos de entrenamiento y prueba
train_data = torch.utils.data.TensorDataset(train_inputs['input_ids'], train_inputs['attention_mask'], torch.tensor(df_train['tipo_num'].tolist()))
test_data = torch.utils.data.TensorDataset(test_inputs['input_ids'], test_inputs['attention_mask'], torch.tensor(df_test['tipo_num'].tolist()))

# Definir el optimizador y la función de pérdida
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

# Entrenamiento del modelo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.train()

train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=16, shuffle=True)

for epoch in range(5):  # Número de épocas de entrenamiento
    total_loss = 0

    for batch in train_dataloader:
        input_ids, attention_mask, labels = tuple(t.to(device) for t in batch)

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    print("Epoch:", epoch + 1, "Loss:", total_loss)

# Evaluación del modelo
model.eval()
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False)

with torch.no_grad():
    predictions = []
    true_labels = []

    for batch in test_dataloader:
        input_ids, attention_mask, labels = tuple(t.to(device) for t in batch)

        outputs = model(input_ids, attention_mask=attention_mask)

        _, predicted_labels = torch.max(outputs.logits, dim=1)

        predictions.extend(predicted_labels.tolist())
        true_labels.extend(labels.tolist())

accuracy = accuracy_score(true_labels, predictions)
print("Precisión:", accuracy)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/310 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/486k [00:00<?, ?B/s]

Epoch: 1 Loss: 127.50858247280121
Epoch: 2 Loss: 101.14248931407928
Epoch: 3 Loss: 75.36851590871811
Epoch: 4 Loss: 53.40990599989891
Epoch: 5 Loss: 36.84128472208977
Precisión: 0.7678571428571429


In [ ]:
# Guardar el modelo entrenado
ruta_modelo = '/content/modelo'
model.save_pretrained(ruta_modelo)
tokenizer.save_pretrained(ruta_modelo)

('/content/modelo/tokenizer_config.json',
 '/content/modelo/special_tokens_map.json',
 '/content/modelo/vocab.txt',
 '/content/modelo/added_tokens.json')

In [ ]:
#Cargar el modelo entrenado
ruta_modelo = '/content/modelo'
Modelo_TF = BertForSequenceClassification.from_pretrained(ruta_modelo)
tokenizer_TF = BertTokenizer.from_pretrained(ruta_modelo)

In [ ]:
# Calcular la precisión por clase
unique_classes = df_dialogo['tipo_num'].unique()

for class_label in unique_classes:
    # Filtrar los datos por clase
    class_data = df_dialogo[df_dialogo['tipo_num'] == class_label]

    # Preparar los datos de la clase para evaluar
    tokens = tokenizer_TF.batch_encode_plus(
        class_data['palabras'].tolist(),
        truncation=True,
        padding=True,
        return_tensors='pt'
    )

    inputs = tokens['input_ids']
    attention_mask = tokens['attention_mask']
    labels = class_data['tipo_num'].tolist()

    # Pasar los datos de la clase por el modelo
    with torch.no_grad():
        outputs = Modelo_TF(inputs, attention_mask=attention_mask)

    predicted_labels = outputs.logits.argmax(dim=1).tolist()

    # Calcular la precisión para la clase
    accuracy = accuracy_score(labels, predicted_labels)
    print(f"Precisión por clase {df_dialogo[df_dialogo.tipo_num == class_label]['tipo'].unique()[0]}: {accuracy}")

Precisión por clase Identidad: 0.9259259259259259
Precisión por clase Nombre: 0.8333333333333334
Precisión por clase Despedida: 0.9074074074074074
Precisión por clase Funcion: 0.9315068493150684
Precisión por clase Agradecimiento: 0.9242424242424242
Precisión por clase Usuario: 0.7567567567567568
Precisión por clase Sentimiento: 0.9295774647887324
Precisión por clase Origen: 0.96
Precisión por clase Continuacion: 0.9032258064516129
Precisión por clase Saludos: 0.9777777777777777
Precisión por clase Error: 0.8333333333333334
Precisión por clase Aprendizaje: 0.6046511627906976
Precisión por clase Otros: 0.9814814814814815
Precisión por clase Edad: 0.8387096774193549
Precisión por clase Contacto: 0.7931034482758621


In [ ]:
# Procesar nueva frase
frase = ' '.join(normalizar('donde vives?'))

# Tokenizar la frase de entrada
tokens = tokenizer_TF.encode_plus(
    frase,
    add_special_tokens=True,
    max_length=128,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)

# Obtener los input_ids y attention_mask
input_ids = tokens['input_ids']
attention_mask = tokens['attention_mask']

# Realizar la predicción
with torch.no_grad():
    outputs = Modelo_TF(input_ids, attention_mask)

# Obtener las etiquetas predichas
etiquetas_predichas = torch.argmax(outputs.logits, dim=1)

# Decodificar las etiquetas predichas
etiquetas_decodificadas = etiquetas_predichas.tolist()

diccionario = {0: 'Agradecimiento',1: 'Aprendizaje',2: 'Contacto',3: 'Continuacion',
               4: 'Despedida',5: 'Edad',6: 'Error',7: 'Funcion',8: 'Identidad',
               9: 'Nombre',10: 'Origen',11: 'Otros',12: 'Saludos',13: 'Sentimiento',
               14: 'Usuario'}
llave_buscada = etiquetas_decodificadas[0]
clase_encontrada = diccionario[llave_buscada]
print("La frase", frase, "se clasifica como: ", clase_encontrada)

La frase donde vivir se clasifica como:  Contacto
